In [ ]:

#  This cell will suppress warning that the MLP routine did not "settle"
#

import warnings
warnings.filterwarnings('ignore')

print("--")


In [ ]:

#  This NoteBook contains code to run MLPClassifier against MNist
#



#  Step 00: Setup

In [ ]:

#  Largely code to control how print statements and related work
#

%load_ext autoreload
%autoreload 2

%xmode Minimal


#  Setting display options 
#
import pandas as pd
pd.set_option("display.width", 640)
   #
import numpy as np
np.set_printoptions(edgeitems = 30, linewidth = 100000, 
   formatter = dict(float = lambda x: "%.3g" % x))

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

   ###
    
from tabulate import tabulate
#
#  How to use tabulate-
#
#  l_result = [{ "col1": 20, "col2": 30}]
#  #
#  print(tabulate(l_result, headers='keys', tablefmt='psql', showindex=False))

print("--")


#  Step A1:  MNist Data load, encode, other

In [ ]:

#  We will be using Keras, so pip install it inside the Jupyter NoteBook container ..
#

l_package1 = "keras"
l_package2 = "tensorflow"
    
def my_func(arg1):
    
   import sys
   import subprocess
    
   subprocess.check_call([sys.executable, "-m", "pip", "install", arg1 ])
      #
   print("From each host ..")
      #
   return


   ##########################################
    

print("Install Python Pip packages on Jupyter container ..")
   #
my_return = my_func(l_package1)
my_return = my_func(l_package2)
print()
    

#  Use this if installing o nthe KGIP worker nodes ..
#
#  print("Install Python Pip packages on KGIP worker node containers ..")
#     # 
#  my_return = my_graph.run(lambda g: my_func(l_package))
#  print()
    
print("--")


In [ ]:

#  Intead of loading MNist from disk, we load it from the Keras library ..
#

import keras
from keras.datasets import mnist


np_mnist = {}
   #
(np_mnist["train"], np_mnist["train_label"]), (np_mnist["test"], np_mnist["test_label"]) = mnist.load_data()


#  train and test ccome in as an array [(n), 28, 28] where n == 60000 for train,
#  and 100000 for test
#
#  We need that 28*28 as a vector, so ..
#
np_mnist["train_v"] = np_mnist["train"].reshape(-1, 28*28)
np_mnist["test_v"]  = np_mnist["test" ].reshape(-1, 28*28)


print("Train shape ................ %s" % (str(np_mnist["train"].shape)))
print("Train label shape .......... %s" % (str(np_mnist["train_label"].shape)))
   #
print("Test  shape ................ %s" % (str(np_mnist["test"].shape)))
print("Test  label shape .......... %s" % (str(np_mnist["test_label"].shape)))
   #
print("Train vector shape ......... %s" % (str(np_mnist["train_v"].shape)))
print("Test  vector shape ......... %s" % (str(np_mnist["test_v" ].shape)))
   #
print()


#  tabulate() displays poorly with this wide data. Straight up print() works well.
#
# print(tabulate(np_mnist["train"][0:2], headers='keys', tablefmt='psql', showindex=False))
print(np_mnist["train"][0:2])
print("Number of rows: %d" % (len(np_mnist["train"])))
   #
print()

print(np_mnist["train_label"][0:2])
print("Number of rows: %d" % (len(np_mnist["train_label"])))
   #
print()


print(np_mnist["test"][0:2])
print("Number of rows: %d" % (len(np_mnist["test"])))
   #
print()

print(np_mnist["test_label"][0:2])
print("Number of rows: %d" % (len(np_mnist["test_label"])))
   #
print()

    
print("--")


In [ ]:

#  Sample results
#
#     Train shape ................ (60000, 28, 28)
#     Train label shape .......... (60000,)
#     Test  shape ................ (10000, 28, 28)
#     Test  label shape .......... (10000,)
#     Train vector shape ......... (60000, 784)
#     Test  vector shape ......... (10000, 784)
#     
#     [[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136 175  26 166 255 247 127   0   0   0   0]
#       [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253 225 172 253 242 195  64   0   0   0   0]
#       [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251  93  82  82  56  39   0   0   0   0   0]
#       [  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 182 247 241   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0  80 156 107 253 253 205  11   0  43 154   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0  14   1 154 253  90   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0 139 253 190   2   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0  11 190 253  70   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0  35 241 225 160 108   1   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0  81 240 253 253 119  25   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0  45 186 253 253 150  27   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  16  93 252 253 187   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 249 253 249  64   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0  46 130 183 253 253 207   2   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0  39 148 229 253 253 253 250 182   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0  24 114 221 253 253 253 253 201  78   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0  23  66 213 253 253 253 253 198  81   2   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0  18 171 219 253 253 253 253 195  80   9   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0  55 172 226 253 253 253 253 244 133  11   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0 136 253 253 253 212 135 132  16   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
#     
#      [[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  51 159 253 159  50   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0  48 238 252 252 252 237   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0  54 227 253 252 239 233 252  57   6   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0  10  60 224 252 253 252 202  84 252 253 122   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0 163 252 252 252 253 252 252  96 189 253 167   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0  51 238 253 253 190 114 253 228  47  79 255 168   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0  48 238 252 252 179  12  75 121  21   0   0 253 243  50   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0  38 165 253 233 208  84   0   0   0   0   0   0 253 252 165   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   7 178 252 240  71  19  28   0   0   0   0   0   0 253 252 195   0   0   0   0   0]
#       [  0   0   0   0   0   0   0  57 252 252  63   0   0   0   0   0   0   0   0   0 253 252 195   0   0   0   0   0]
#       [  0   0   0   0   0   0   0 198 253 190   0   0   0   0   0   0   0   0   0   0 255 253 196   0   0   0   0   0]
#       [  0   0   0   0   0   0  76 246 252 112   0   0   0   0   0   0   0   0   0   0 253 252 148   0   0   0   0   0]
#       [  0   0   0   0   0   0  85 252 230  25   0   0   0   0   0   0   0   0   7 135 253 186  12   0   0   0   0   0]
#       [  0   0   0   0   0   0  85 252 223   0   0   0   0   0   0   0   0   7 131 252 225  71   0   0   0   0   0   0]
#       [  0   0   0   0   0   0  85 252 145   0   0   0   0   0   0   0  48 165 252 173   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0  86 253 225   0   0   0   0   0   0 114 238 253 162   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0  85 252 249 146  48  29  85 178 225 253 223 167  56   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0  85 252 252 252 229 215 252 252 252 196 130   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0  28 199 252 252 253 252 252 233 145   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0  25 128 252 253 252 141  37   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]]
#     Number of rows: 60000
#     
#     [5 0]
#     Number of rows: 60000
#     
#     [[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0  84 185 159 151  60  36   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0 222 254 254 254 254 241 198 198 198 198 198 198 198 198 170  52   0   0   0   0   0   0]
#       [  0   0   0   0   0   0  67 114  72 114 163 227 254 225 254 254 254 250 229 254 254 140   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0  17  66  14  67  67  67  59  21 236 254 106   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  83 253 209  18   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  22 233 255  83   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 129 254 238  44   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  59 249 254  62   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 133 254 187   5   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   9 205 248  58   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 126 254 182   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0  75 251 240  57   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0  19 221 254 166   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   3 203 254 219  35   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0  38 254 254  77   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0  31 224 254 115   1   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0 133 254 254  52   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0  61 242 254 254  52   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0 121 254 254 219  40   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0 121 254 207  18   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
#     
#      [[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0 116 125 171 255 255 150  93   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0 169 253 253 253 253 253 253 218  30   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0 169 253 253 253 213 142 176 253 253 122   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0  52 250 253 210  32  12   0   6 206 253 140   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0  77 251 210  25   0   0   0 122 248 253  65   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0  31  18   0   0   0   0 209 253 253  65   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0 117 247 253 198  10   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0  76 247 253 231  63   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0 128 253 253 144   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0 176 246 253 159  12   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0  25 234 253 233  35   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0 198 253 253 141   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0  78 248 253 189  12   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0  19 200 253 253 141   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0 134 253 253 173  12   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0 248 253 253  25   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0 248 253 253  43  20  20  20  20   5   0   5  20  20  37 150 150 150 147  10   0]
#       [  0   0   0   0   0   0   0   0 248 253 253 253 253 253 253 253 168 143 166 253 253 253 253 253 253 253 123   0]
#       [  0   0   0   0   0   0   0   0 174 253 253 253 253 253 253 253 253 253 253 253 249 247 247 169 117 117  57   0]
#       [  0   0   0   0   0   0   0   0   0 118 123 123 123 166 253 253 253 155 123 123  41   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
#       [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]]
#     Number of rows: 10000
#     
#     [7 2]
#     Number of rows: 10000


#  Step A2: MNist train, test .. (All)

In [ ]:


import numpy as np
import time
from sklearn.neural_network import MLPClassifier

def run(x_train, y_train, x_test, y_test, clf):
   s = time.time()
   clf.fit(x_train, y_train)
   e = time.time()-s
   loss = clf.loss_
   weights = clf.coefs_
   biases = clf.intercepts_
   params = 0
   for w in weights:
      params += w.shape[0]*w.shape[1]
   for b in biases:
      params += b.shape[0]
   return [clf.score(x_test, y_test), loss, params, e]


def nn(layers, act):
   return MLPClassifier(solver="sgd", verbose=False, tol=1e-8,
          nesterovs_momentum=False, early_stopping=False,
          learning_rate_init=0.001, momentum=0.9, max_iter=200,
          hidden_layer_sizes=layers, activation=act)


def main():
    
   #  x_train = np.load("../data/mnist/mnist_train_vectors.npy").astype("float64")/256.0
   #  y_train = np.load("../data/mnist/mnist_train_labels.npy")
   #  x_test = np.load("../data/mnist/mnist_test_vectors.npy").astype("float64")/256.0
   #  y_test = np.load("../data/mnist/mnist_test_labels.npy")
    
   #  In scope at this point,
   #
   #  np_mnist["train"]
   #  np_mnist["train_v"]
   #  np_mnist["train_label"]
   #  np_mnist["test"]
   #  np_mnist["test_v"]
   #  np_mnist["test_label"]
    
   x_train = np_mnist["train_v"].astype("float64") / 265.0 
   y_train = np_mnist["train_label"]
   x_test  = np_mnist["test_v" ].astype("float64") / 265.0 
   y_test  = np_mnist["test_label"]

    
   N = 1000
   x_train = x_train[:N]
   y_train = y_train[:N]
   x_test  = x_test[:N]
   y_test  = y_test[:N]

   layers = [
      (1,), (500,), (800,), (1000,), (2000,), (3000,),
      (1000,500), (3000,1500),
      (2,2,2), (1000,500,250), (2000,1000,500),
   ]

   for act in ["relu", "logistic", "tanh"]:
      print("%s:" % act)
      for layer in layers:
         scores = []
         loss = []
         tm = []
         for i in range(10):
            s,l,params,e = run(x_train, y_train, x_test, y_test, nn(layer,act))
            scores.append(s)
            loss.append(l)
            tm.append(e)
         s = np.array(scores)
         l = np.array(loss)
         t = np.array(tm)
         n = np.sqrt(s.shape[0])
         print("    layers: %14s, score= %0.4f +/- %0.4f, loss = %0.4f +/- %0.4f (params = %6d, time = %0.2f s)" % \
            (str(layer), s.mean(), s.std()/n, l.mean(), l.std()/n, params, t.mean()))

main()

print("--")

#  Sample output
#
#     relu:
#         layers:           (1,), score= 0.2012 +/- 0.0110, loss = 2.0325 +/- 0.0302 (params =    805, time = 1.52 s)
#         layers:         (500,), score= 0.8272 +/- 0.0018, loss = 0.3507 +/- 0.0008 (params = 397510, time = 13.58 s)
#         layers:         (800,), score= 0.8288 +/- 0.0020, loss = 0.3419 +/- 0.0007 (params = 636010, time = 19.20 s)
#         layers:        (1000,), score= 0.8288 +/- 0.0010, loss = 0.3386 +/- 0.0008 (params = 795010, time = 23.09 s)
#         layers:        (2000,), score= 0.8334 +/- 0.0010, loss = 0.3264 +/- 0.0007 (params = 1590010, time = 43.08 s)
#         layers:        (3000,), score= 0.8358 +/- 0.0011, loss = 0.3231 +/- 0.0004 (params = 2385010, time = 64.09 s)
#         layers:    (1000, 500), score= 0.8394 +/- 0.0008, loss = 0.2357 +/- 0.0009 (params = 1290510, time = 38.16 s)
#         layers:   (3000, 1500), score= 0.8447 +/- 0.0014, loss = 0.2178 +/- 0.0007 (params = 6871510, time = 191.41 s)
#         layers:      (2, 2, 2), score= 0.1613 +/- 0.0150, loss = 2.1752 +/- 0.0414 (params =   1612, time = 0.85 s)
#         layers: (1000, 500, 250), score= 0.8447 +/- 0.0012, loss = 0.1760 +/- 0.0012 (params = 1413260, time = 43.15 s)
#         layers: (2000, 1000, 500), score= 0.8503 +/- 0.0012, loss = 0.1580 +/- 0.0009 (params = 4076510, time = 105.13 s)
#     logistic:
#         layers:           (1,), score= 0.1621 +/- 0.0139, loss = 2.2663 +/- 0.0066 (params =    805, time = 1.62 s)
#         layers:         (500,), score= 0.6357 +/- 0.0022, loss = 1.5699 +/- 0.0022 (params = 397510, time = 15.40 s)
#         layers:         (800,), score= 0.6426 +/- 0.0021, loss = 1.5305 +/- 0.0014 (params = 636010, time = 21.53 s)
#         layers:        (1000,), score= 0.6440 +/- 0.0019, loss = 1.5077 +/- 0.0025 (params = 795010, time = 25.98 s)
#         layers:        (2000,), score= 0.6569 +/- 0.0029, loss = 1.4567 +/- 0.0014 (params = 1590010, time = 46.35 s)
#         layers:        (3000,), score= 0.6581 +/- 0.0021, loss = 1.4314 +/- 0.0014 (params = 2385010, time = 68.05 s)
#         layers:    (1000, 500), score= 0.2098 +/- 0.0025, loss = 2.2723 +/- 0.0003 (params = 1290510, time = 42.37 s)
#         layers:   (3000, 1500), score= 0.1795 +/- 0.0104, loss = 2.2666 +/- 0.0003 (params = 6871510, time = 205.03 s)
#         layers:      (2, 2, 2), score= 0.1098 +/- 0.0042, loss = 2.3003 +/- 0.0003 (params =   1612, time = 0.82 s)
#         layers: (1000, 500, 250), score= 0.1040 +/- 0.0027, loss = 2.2985 +/- 0.0001 (params = 1413260, time = 12.53 s)
#         layers: (2000, 1000, 500), score= 0.1017 +/- 0.0026, loss = 2.2989 +/- 0.0000 (params = 4076510, time = 17.35 s)
#     tanh:
#         layers:           (1,), score= 0.2291 +/- 0.0102, loss = 2.0051 +/- 0.0098 (params =    805, time = 1.55 s)
#         layers:         (500,), score= 0.8263 +/- 0.0016, loss = 0.3435 +/- 0.0006 (params = 397510, time = 13.09 s)
#         layers:         (800,), score= 0.8289 +/- 0.0012, loss = 0.3281 +/- 0.0011 (params = 636010, time = 18.77 s)
#         layers:        (1000,), score= 0.8296 +/- 0.0021, loss = 0.3217 +/- 0.0005 (params = 795010, time = 21.21 s)
#         layers:        (2000,), score= 0.8316 +/- 0.0011, loss = 0.3060 +/- 0.0008 (params = 1590010, time = 39.46 s)
#         layers:        (3000,), score= 0.8354 +/- 0.0010, loss = 0.2994 +/- 0.0005 (params = 2385010, time = 57.70 s)
#         layers:    (1000, 500), score= 0.8440 +/- 0.0010, loss = 0.2246 +/- 0.0008 (params = 1290510, time = 36.16 s)
#         layers:   (3000, 1500), score= 0.8468 +/- 0.0010, loss = 0.1922 +/- 0.0006 (params = 6871510, time = 188.91 s)
#         layers:      (2, 2, 2), score= 0.2321 +/- 0.0132, loss = 1.9969 +/- 0.0270 (params =   1612, time = 0.82 s)
#         layers: (1000, 500, 250), score= 0.8431 +/- 0.0015, loss = 0.1673 +/- 0.0007 (params = 1413260, time = 40.96 s)
#         layers: (2000, 1000, 500), score= 0.8450 +/- 0.0010, loss = 0.1401 +/- 0.0005 (params = 4076510, time = 102.35 s)
    